In [74]:
import numpy as np
from matplotlib import pyplot as plt
import mne
import tensorflow as tf
import os

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, Permute, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import SeparableConv2D, DepthwiseConv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import SpatialDropout2D
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.layers import Input, Flatten
from tensorflow.keras.constraints import max_norm
from tensorflow.keras import backend as K
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import TopKCategoricalAccuracy, AUC, Precision, Recall
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

In [75]:
N_FILES = 3

WHERE = "kaggle"
if WHERE=="colab":
    BCICIV_PATH = "/content/BCICIV_2a_gdf"
    BATCH_SIZE = 32
elif WHERE=="kaggle":
    BCICIV_PATH = "../input/eeg-data-augmentation/BCICIV_2a_gdf"
    BATCH_SIZE = 64
elif WHERE=="home":
    PATH_LABELS = "../toy_data/BCICIV_2a_gdf"
    BATCH_SIZE = 4
SHUFFLE_BUFFER_SIZE = BATCH_SIZE * 2 

EPOCHS = 5
LEARNING_RATE = 1e-3

In [76]:
def load_data():
    files = []
    cpt = 0
    for filename in os.listdir(BCICIV_PATH):
        if cpt >= N_FILES:
            break
        files.append(mne.io.read_raw_gdf(f'{BCICIV_PATH}/{filename}'))
        cpt += 1
    return files

## NEED TO DO A STRATIFIED SPLIT ACCROSS THE DIFFERENT USER OF THE EXPERIMENT !!!
def preprocess_data(data):
#     print(data.info)
#     data.compute_psd().plot()
#     plt.show()
    preprocessed_data = data[0].get_data()
    for d in data[1:]:
        preprocessed_data = np.concatenate((preprocessed_data, d.get_data()), axis=1)
    n_chans, n_steps = preprocessed_data.shape
    print("--> This dataset contains", n_chans, "channels and", n_steps, "steps")
    print("--> 25 channels: 22 EEG and 3 EOG")
    return preprocessed_data
#     return X_train, y_train, X_valid, y_valid, X_test, y_test 

def visualize_data():
    pass

def make_tf_dataset(X_train, y_train, X_valid, y_valid, X_test, y_test):
    train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
    valid_dataset = tf.data.Dataset.from_tensor_slices((X_valid, y_valid))
    test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

    train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
    valid_dataset = valid_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
    test_dataset = test_dataset.batch(BATCH_SIZE)

    return train_dataset, valid_dataset, test_dataset

In [77]:
data = load_data()
preprocess_data(data)
# X_train, y_train, X_valid, y_valid, X_test, y_test = preprocess_data(data)

Extracting EDF parameters from /kaggle/input/eeg-data-augmentation/BCICIV_2a_gdf/A01T.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
EEG-Fz, EEG, EEG, EEG, EEG, EEG, EEG, EEG-C3, EEG, EEG-Cz, EEG, EEG-C4, EEG, EEG, EEG, EEG, EEG, EEG, EEG, EEG-Pz, EEG, EEG, EOG-left, EOG-central, EOG-right
Creating raw.info structure...
Extracting EDF parameters from /kaggle/input/eeg-data-augmentation/BCICIV_2a_gdf/A04E.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
EEG-Fz, EEG, EEG, EEG, EEG, EEG, EEG, EEG-C3, EEG, EEG-Cz, EEG, EEG-C4, EEG, EEG, EEG, EEG, EEG, EEG, EEG, EEG-Pz, EEG, EEG, EOG-left, EOG-central, EOG-right
Creating raw.info structure...
Extracting EDF parameters from /kaggle/input/eeg-data-augmentation/BCICIV_2a_gdf/A08T.gdf...
GDF file detected
Setting channel info structure...
Could not determi

/opt/conda/lib/python3.7/contextlib.py:119: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)
/opt/conda/lib/python3.7/contextlib.py:119: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)
/opt/conda/lib/python3.7/contextlib.py:119: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


--> This dataset contains 25 channels and 2007845 steps
--> 25 channels: 22 EEG and 3 EOG


array([[-2.11425781e-05, -2.19238281e-05, -1.56250000e-05, ...,
        -8.93554688e-06, -2.05078125e-06, -2.24609375e-06],
       [-2.36816406e-05, -2.39257812e-05, -1.97265625e-05, ...,
        -1.23046875e-05, -5.12695312e-06, -6.64062500e-06],
       [-2.14843750e-05, -2.43164062e-05, -1.88476562e-05, ...,
        -7.03125000e-06, -1.46484375e-06, -2.63671875e-06],
       ...,
       [ 3.17382813e-05,  2.24609375e-05,  3.36914062e-05, ...,
        -2.44140625e-06,  4.39453125e-06, -1.95312500e-06],
       [-1.36718750e-05, -1.31835937e-05, -8.30078125e-06, ...,
        -5.85937500e-06, -9.76562500e-07, -4.39453125e-06],
       [-4.39453125e-05, -4.05273437e-05, -3.85742188e-05, ...,
        -5.37109375e-06,  0.00000000e+00, -3.90625000e-06]])

In [78]:
def get_callbacks():
    return [
        # ModelCheckpoint(
        #     "best_model.h5", save_best_only=True, monitor="loss"
        # ),
        # ReduceLROnPlateau(
        #     monitor="val_top_k_categorical_accuracy",
        #     factor=0.2,
        #     patience=2,
        #     min_lr=0.000001,
        # ),
        EarlyStopping(
            monitor="val_loss",
            min_delta=1e-3,
            patience=10,
            verbose=1,
            mode="auto",
            baseline=None,
            restore_best_weights=True,
        ),
    ]

def EEGNet(n_classes, channels=64, samples=128, kernel_length=64, n_filters1=8, n_filters2=16, depth_multiplier=2, norm_rate=0.25, dropout_rate=0.5, dropoutType="Dropout"):

    if dropoutType == "SpatialDropout2D":
        dropoutType=SpatialDropout2D
    elif dropoutType == "Dropout":
        dropoutType=Dropout

    input = Input(shape=(channels, samples, 1))

    block1 = Conv2D(n_filters1, (1, kernel_length), padding="same", input_shape=(channels, samples, 1), use_bias=False)(input)
    block1 = BatchNormalization()(block1)
    block1 = DepthwiseConv2D((channels, 1), use_bias=False, depth_multiplier=depth_multiplier, depthwise_constraint=max_norm(1.0))(block1)
    block1 = BatchNormalization()(block1)
    block1 = Activation("elu")(block1)
    block1 = AveragePooling2D((1, 4))(block1)
    block1 = dropoutType(dropout_rate)(block1)

    block2 = SeparableConv2D(n_filters2, (1, 16), use_bias=False, padding="same")(block1)
    block2 = BatchNormalization()(block2)
    block2 = Activation("elu")(block2)
    block2 = AveragePooling2D((1, 8))(block2)
    block2 = dropoutType(dropout_rate)(block2)
    block2 = Flatten(name="flatten")(block2)

    classifier = Dense(n_classes, name="dense", kernel_constraint=max_norm(norm_rate))(block2)
    classifier = Activation("softmax", name="softmax")(classifier)

    model = Model(inputs=input, outputs=classifier) 

    optimizer = Adam(amsgrad=True, learning_rate=LEARNING_RATE)
    loss = CategoricalCrossentropy()

    model.compile(
        optimizer=optimizer,
        loss=loss,
        metrics=[
            TopKCategoricalAccuracy(k=3),
            AUC(),
            Precision(),
            Recall(),
        ],
    )

    model.summary()

    return model

In [71]:
def plot_history_metrics(history):
    total_plots = len(history.history)
    cols = total_plots // 2

    rows = total_plots // cols

    if total_plots % cols != 0:
        rows += 1

    pos = range(1, total_plots + 1)
    plt.figure(figsize=(15, 10))
    for i, (key, value) in enumerate(history.history.items()):
        plt.subplot(rows, cols, pos[i])
        plt.plot(range(len(value)), value)
        plt.title(str(key))
    plt.show()

In [72]:
# data = load_data()
# X_train, y_train, X_valid, y_valid, X_test, y_test = preprocess_data(data)
# visualize_data(data)
# train_dataset, valid_dataset, test_dataset = make_tf_dataset(X_train, y_train, X_valid, y_valid, X_test, y_test)

In [73]:
model = EEGNet(n_classes=???)

SyntaxError: invalid syntax (667406098.py, line 1)

In [ ]:
history = model.fit(
    train_dataset,
    epochs=EPOCHS,
    callbacks=get_callbacks(),
    validation_data=valid_dataset,
    class_weight=???,
)

In [ ]:
plot_history_metrics(history)